# Demonstrate Word Embeddings with IMDB movie reviews

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
# Load the IMDB movie reviews dataset

imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

2021-12-11 12:16:35.170691: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Extract the relevant data lists from the training and test data

import numpy as np

train_data , test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s, l in train_data:
    training_sentences.append(s.numpy().decode('utf8'))
    training_labels.append(l.numpy())

for s, l in test_data:
    testing_sentences.append(s.numpy().decode('utf8'))
    testing_labels.append(l.numpy())

# For trainings , our labels are expected to be numpy arrays
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

2021-12-11 12:16:35.394164: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


### Tokenize our sentences

In [4]:
# Hyperparameters

vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_token = '<OOV>'

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

# Similarly for testing sequences
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [6]:
# Lets check one sentence - original and decoded from padding

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? a very close and sharp <OOV> of the <OOV> and dynamic emotional world of <OOV> one <OOV> old guy that makes his first experiences in his gay love to an other boy during an vacation with a part of his family br br i liked this film because of his <OOV> clear and <OOV> storytelling with all this sound close ups and quiet moments <OOV> had been full of <OOV> <OOV> br br
A very close and sharp discription of the bubbling and dynamic emotional world of specialy one 18year old guy, that makes his first experiences in his gay love to an other boy, during an vacation with a part of his family.<br /><br />I liked this film because of his extremly clear and surrogated storytelling , with all this "Sound-close-ups" and quiet moments wich had been full of intensive moods.<br /><br />


### Now we will build and train the model

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation=tf.nn.relu),
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Lets run the training
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 3s 3ms/step - loss: 0.5981 - accuracy: 0.6403 - val_loss: 0.3393 - val_accuracy: 0.8521
Epoch 2/10
782/782 [==============================] - 2s 3ms/step - loss: 0.2359 - accuracy: 0.9121 - val_loss: 0.3675 - val_accuracy: 0.8404
Epoch 3/10
782/782 [==============================] - 2s 3ms/step - loss: 0.0976 - accuracy: 0.9762 - val_loss: 0.4681 - val_accuracy: 0.8228
Epoch 4/10
782/782 [==============================] - 2s 2ms/step - loss: 0.0247 - accuracy: 0.9972 - val_loss: 0.5238 - val_accuracy: 0.8273
Epoch 5/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0068 - accuracy: 0.9995 - val_loss: 0.5889 - val_accuracy: 0.8281
Epoch 6/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.6374 - val_accuracy: 0.8288
Epoch 7/10
782/782 [==============================] - 3s 4ms/step - loss: 9.6001e-04 - accuracy: 1.0000 - val_loss: 0.6845 - val_accuracy: 0.8288
Ep

In [9]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)  # We have a vocab_size of 10000 and embedding dimension of 16

(10000, 16)


### Now we will write the vectors and metadata onto a file

We will then load the tsv files and view them on
https://projector.tensorflow.org

In [10]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()